In [1]:
import pandas as pd

In [2]:
# Чтение и запись данных в датафрейм
df_zones = pd.read_csv('place_zone_coordinates.csv')
df_users = pd.read_csv('user_coordinates.csv')

In [3]:
df_zones

,place_id,loc_lat,loc_lon,point_number
0,1,55.747022,37.787073,0
1,1,55.751713,37.784328,1
2,1,55.753878,37.777638,2
3,1,55.751031,37.779351,3
4,2,55.803885,37.458311,0
5,2,55.808677,37.464054,1
6,2,55.809763,37.461314,2
7,2,55.810840,37.458654,3


In [4]:
df_users

,user_id,loc_lat,loc_lon
0,1,55.737564,37.345186
1,2,56.234564,37.234590
2,3,55.234578,36.295745


In [5]:
# Функция для определения находится ли точка (x, y) внутри полигона с координатами (x_p, y_p); x_p, y_p - м.б. заданы списком.
# Метод трассировки луча.

def in_polygon(x, y, x_p, y_p):
    is_in = 0
    
    for i in range(len(x_p)):
        if (((y_p[i] <= y and y < y_p[i-1]) or (y_p[i-1] <= y and y < y_p[i])) 
            and 
            (x > (x_p[i-1] - x_p[i]) * (y - y_p[i]) / (y_p[i-1] - y_p[i]) + x_p[i])):
            is_in = 1 - is_in   
    
    return bool(is_in)


# Проверка
print(in_polygon(1, 1, (10, -10, 10, -10), (10, 10, -10, -10)))

True


In [6]:
# Создание датафрейма для записи результата
df_result = pd.DataFrame(columns = ['id', 'number_of_places_available'])
ind = 0

for user in df_users.user_id:
    
    user_loc_lat = df_users[df_users.user_id == user].loc_lat.values
    user_loc_lon = df_users[df_users.user_id == user].loc_lon.values
    
    count = 0
    
    for place in df_zones['place_id'].unique():
        
        x = df_zones[df_zones.place_id == place].loc_lat.values
        y = df_zones[df_zones.place_id == place].loc_lon.values
        
        # Находится ли координата (user_loc_lat, user_loc_lon) в полигоне x, y
        is_in = in_polygon(user_loc_lat, user_loc_lon, x, y)
        
        if is_in:
            count += 1
    
    # Запись результата
    df_result.loc[ind, 'id'] = user
    df_result.loc[ind, 'number_of_places_available'] = count
    ind += 1
        
# Запись данных в csv  
df_result.to_csv('result.csv', index=False)

df_result

,id,number_of_places_available
0,1.0,0.0
1,2.0,0.0
2,3.0,0.0
